In [ ]:
%matplotlib ipympl
SCOPETYPE = "OPENADC"
PLATFORM = "CWHUSKY"
# SS_VER = "SS_VER_2_1"

%run "Setup_Generic.ipynb"


import chipwhisperer as cw
from chipwhisperer.capture.scopes.OpenADC import OpenADC
from chipwhisperer.capture.targets.SimpleSerial2 import SimpleSerial2
from chipwhisperer.capture.api.programmers import XMEGAProgrammer
import matplotlib.pyplot as plt
import time
from pprint import pprint
import numpy as np
from typing import Optional, List, Set, Any
print(type(scope))
print(type(target))
print(type(prog))
scope: OpenADC = scope
target: SimpleSerial2 = target
prog: XMEGAProgrammer = prog

print("Scope:\n", scope)

In [ ]:
scope.cglitch_setup()


In [ ]:
refresh_rate = 200e6

scope.adc.timeout = 5
scope.adc.samples = 118_000
scope.clock.clkgen_src = "system" 
scope.clock.clkgen_freq = refresh_rate
scope.clock.adc_mul = 1
print("'Real freq': scope.clock.adc_freq:", scope.clock.adc_freq)
if scope.clock.adc_freq is not None:
    print(scope.clock.adc_freq / 1e6, "MHz")

scope.io.glitch_hp = True
scope.io.glitch_lp = True

In [ ]:
from chipwhisperer.common.results.glitch import GlitchController

gc = GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset"])
gc.clear()
gc.display_stats()


In [ ]:
import datetime
import sys

LOG_FILE = "glitch_log.txt"

def log_print(*args, **kwargs):
    timestamp = datetime.datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
    message = " ".join(str(a) for a in args)
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(f"{timestamp} {message}\n")
    print(f"{timestamp} {message}", **kwargs)

In [ ]:
import paramiko
import threading
import time
import socket
import math

traces = []

# ===== SSH SETUP =====
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

TARGET_IP = "172.20.10.2"
USERNAME = "test"
PASSWORD = "test"

def connect_ssh():
    global client
    reset_duration = 0
    while True:
        try:
            client.connect(
                hostname=TARGET_IP,
                port=22,
                username=USERNAME,
                password=PASSWORD,
                timeout=5
            )
            log_print("[+] Connected to target")
            return
        except (
            paramiko.AuthenticationException,
            paramiko.SSHException,
            socket.error
        ) as e:
            log_print(f"[!] Target not reachable, waiting for reboot... (total time: {reset_duration}s)")
            print("Error type:", type(e))
            print("Error text:", e)
            time.sleep(5)
            reset_duration += 5
            continue

def reconnect_target():
    global client
    try:
        client.close()
    except Exception:
        pass
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    connect_ssh()

def is_ssh_alive():
    try:
        stdin, stdout, stderr = client.exec_command("echo alive", timeout=5)
        result = stdout.read().decode().strip()
        return result == "alive"
    except Exception:
        return False


def run_remote_command():
    time.sleep(0.2)
    try:
        stdin, stdout, stderr = client.exec_command(
            'cd trigger-test/freqs-2147MHz-sync-single-core/; ./main-200MHz-io',
            timeout = 5.0
        )
        # print("Target executing program")
        output = stdout.read().decode('utf-8')
        err = stderr.read().decode('utf-8')

        if output.startswith("aaaaaaaaa"):
            gc.add("normal")
        else:
            gc.add("success")
            log_print("HIT GLITCH:", output)

    except (paramiko.SSHException, socket.error):
        log_print("[!] Lost SSH connection during execution.")
        gc.add("reset")  # optional — mark this glitch as a crash


connect_ssh()

gc.set_range("offset", 0, scope.glitch.phase_shift_steps)
gc.set_range("width", scope.glitch.phase_shift_steps, scope.glitch.phase_shift_steps)

gc.set_global_step([400])

gc.set_range("ext_offset", 17_000, 28_000)
gc.set_step("ext_offset", 1)


# scope.glitch.repeat = 8192
scope.glitch.repeat = 8192

for glitch_settings in gc.glitch_values():
    
    scope.glitch.offset = glitch_settings[1]
    scope.glitch.width = glitch_settings[0]
    scope.glitch.ext_offset = glitch_settings[2]
    log_print(f"Offset: {scope.glitch.offset}, Width: {scope.glitch.width}, Ext_offset: {scope.glitch.ext_offset}, Repeat: {scope.glitch.repeat}")
    print(f"scope.adc.trig_count: {scope.adc.trig_count}")

    # If the target is offline, pause glitching until it's back
    while True:
        try:
            if is_ssh_alive():
                break
            else:
                log_print("[!] SSH inactive, waiting...")
                reconnect_target()
        except Exception as e:
            log_print("Found unhandled error case:", type(e), e)
            reconnect_target()

    # Normal glitch process
    scope.arm()
    # print("Starting remote process")
    thread = threading.Thread(target=run_remote_command)
    thread.start()

    # print("Listening to target")
    ret = scope.capture()
    # print("Finished listening to target")

    thread.join()

    # traces.append(scope.get_last_trace())
    # print("Target finished")

    if ret:
        log_print('Timeout - no trigger')
        gc.add("reset")


In [ ]:
for t in traces:
    print(t)

In [ ]:
%matplotlib ipympl
plt.figure()
for t in traces:
    plt.plot(t)
plt.grid()
plt.show()



In [ ]:


%matplotlib ipympl
for t in traces[:5]:
    print(t)
    plt.figure()
    plt.plot(t)
    plt.grid()
    plt.show()
